In [ ]:
import numpy as np
from sklearn.cluster import KMeans

In [ ]:
class FeatureGeneratorClustering:
    def __init__(self, cleaned_tweets_word_list, w2vmodel, num_features):
        self.__cleaned_tweets_word_list = cleaned_tweets_word_list
        self.__w2vmodel = w2vmodel
        self.__num_features = num_features
        self.__features_vec = None
        
        # Clusterizando
        word_vectors = w2vmodel.wv.syn0
        num_clusters = int(word_vectors.shape[0]/5)
        kmeans_clustering = KMeans(n_clusters = num_clusters)
        idx = kmeans_clustering.fit_predict(word_vectors)
        self.__word_centroid_map = dict(zip(w2vmodel.wv.index2word, idx))

    def gen_features_vec(self):
        cont = 0
        num_centroids = max(self.__word_centroid_map.values()) + 1
        self.__features_vec = np.zeros((len(self.__cleaned_tweets_word_list), num_centroids))
        for tweet in self.__cleaned_tweets_word_list:
            self.__features_vec[cont] = self.__make_features_vec(tweet)
            cont += 1
            
    def __make_features_vec(self, tweet):
        num_centroids = max(self.__word_centroid_map.values()) + 1
        bag_of_centroids = np.zeros(num_centroids, dtype="float32")
        for word in tweet:
            if word in self.__word_centroid_map:
                index = self.__word_centroid_map[word]
                bag_of_centroids[index] += 1
        return bag_of_centroids
    
    def get_features_vec(self):
        return self.__features_vec
    
    def print_clusters(self, n):
        for cluster in range(n):
            # Print the cluster number  
            print(" [->] Cluster {}:".format(cluster))
            # Find all of the words for that cluster number, and print them out
            words = []
            for i in range(len(self.__word_centroid_map.values())):
                if(list(self.__word_centroid_map.values())[i] == cluster):
                    words.append(list(self.__word_centroid_map.keys())[i])
            print(words)
